# بناء نموذج الانحدار: إعداد البيانات وتصويرها

## **الانحدار الخطي للقرع - الدرس الثاني**
#### المقدمة

الآن بعد أن أصبحت مجهزًا بالأدوات اللازمة لبدء بناء نماذج تعلم الآلة باستخدام Tidymodels وTidyverse، أصبحت جاهزًا لبدء طرح الأسئلة على بياناتك. أثناء العمل مع البيانات وتطبيق حلول تعلم الآلة، من المهم جدًا فهم كيفية طرح السؤال الصحيح لفتح إمكانيات مجموعة البيانات الخاصة بك بشكل صحيح.

في هذا الدرس، ستتعلم:

- كيفية إعداد بياناتك لبناء النماذج.

- كيفية استخدام `ggplot2` لتصور البيانات.

السؤال الذي تحتاج إلى الإجابة عليه سيحدد نوع خوارزميات تعلم الآلة التي ستستخدمها. كما أن جودة الإجابة التي تحصل عليها تعتمد بشكل كبير على طبيعة بياناتك.

دعونا نرى ذلك من خلال تمرين عملي.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>عمل فني بواسطة @allison_horst</figcaption>


## 1. استيراد بيانات القرع واستدعاء Tidyverse

سنحتاج إلى الحزم التالية لتقسيم وتحليل هذا الدرس:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) هو [مجموعة من حزم R](https://www.tidyverse.org/packages) مصممة لجعل علم البيانات أسرع وأسهل وأكثر متعة!

يمكنك تثبيتها باستخدام:

`install.packages(c("tidyverse"))`

السكربت أدناه يتحقق مما إذا كانت لديك الحزم المطلوبة لإكمال هذا الوحدة ويقوم بتثبيتها إذا كانت بعض الحزم مفقودة.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

الآن، دعونا نقوم بتشغيل بعض الحزم وتحميل [البيانات](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) المقدمة لهذا الدرس!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

نظرة سريعة باستخدام `glimpse()` تكشف فورًا وجود فراغات ومزيج من النصوص (`chr`) والبيانات الرقمية (`dbl`). العمود `Date` هو من نوع النصوص، وهناك أيضًا عمود غريب يسمى `Package` يحتوي على بيانات مختلطة بين `sacks`، `bins`، وقيم أخرى. في الواقع، البيانات تبدو فوضوية بعض الشيء 😤.

في الحقيقة، من النادر جدًا أن تحصل على مجموعة بيانات جاهزة تمامًا للاستخدام مباشرة في إنشاء نموذج تعلم آلي. لكن لا تقلق، في هذا الدرس ستتعلم كيفية تجهيز مجموعة بيانات خام باستخدام مكتبات R القياسية 🧑‍🔧. كما ستتعلم تقنيات متنوعة لتصور البيانات. 📈📊
<br>

> تذكير سريع: مشغل الأنابيب (`%>%`) ينفذ العمليات بتسلسل منطقي عن طريق تمرير كائن إلى الأمام داخل دالة أو تعبير استدعاء. يمكنك التفكير في مشغل الأنابيب كأنه يقول "ثم" في الكود الخاص بك.


## 2. التحقق من البيانات المفقودة

أحد أكثر المشاكل شيوعًا التي يواجهها علماء البيانات هي البيانات غير المكتملة أو المفقودة. يمثل R القيم المفقودة أو غير المعروفة باستخدام قيمة خاصة: `NA` (غير متوفرة).

كيف يمكننا معرفة ما إذا كان إطار البيانات يحتوي على قيم مفقودة؟
<br>
- إحدى الطرق المباشرة هي استخدام وظيفة R الأساسية `anyNA` التي تُرجع الكائنات المنطقية `TRUE` أو `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

هناك بعض البيانات المفقودة! هذه نقطة جيدة للبدء.

-   طريقة أخرى هي استخدام الدالة `is.na()` التي تشير إلى العناصر المفقودة في الأعمدة الفردية باستخدام القيمة المنطقية `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

حسنًا، لقد أنجزت المهمة ولكن مع إطار بيانات كبير مثل هذا، سيكون من غير الفعّال ومن المستحيل عمليًا مراجعة جميع الصفوف والأعمدة بشكل فردي 😴.

-   طريقة أكثر سهولة ستكون حساب مجموع القيم المفقودة لكل عمود:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

أفضل بكثير! هناك بيانات مفقودة، ولكن ربما لن تكون مهمة لهذه المهمة. دعونا نرى ما ستكشفه المزيد من التحليلات.

> بالإضافة إلى المجموعات الرائعة من الحزم والوظائف، يتميز R بتوثيق جيد جدًا. على سبيل المثال، استخدم `help(colSums)` أو `?colSums` لمعرفة المزيد عن الوظيفة.


## 3. Dplyr: قواعد معالجة البيانات

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>عمل فني بواسطة @allison_horst</figcaption>


<!--![عمل فني بواسطة \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.ar.png)<br/>عمل فني بواسطة \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/)، وهي حزمة ضمن Tidyverse، تمثل قواعد لتنظيم البيانات وتوفر مجموعة متناسقة من الأفعال التي تساعدك على حل التحديات الأكثر شيوعًا في معالجة البيانات. في هذا القسم، سنستكشف بعض الأفعال التي تقدمها dplyr!  
<br>


#### dplyr::select()

`select()` هي دالة في حزمة `dplyr` تساعدك على اختيار الأعمدة التي تريد الاحتفاظ بها أو استبعادها.

لجعل إطار البيانات الخاص بك أسهل في التعامل، قم بحذف بعض الأعمدة باستخدام `select()`، مع الاحتفاظ فقط بالأعمدة التي تحتاجها.

على سبيل المثال، في هذا التمرين، تحليلنا سيتضمن الأعمدة `Package`، `Low Price`، `High Price` و `Date`. دعنا نختار هذه الأعمدة.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` هي دالة في حزمة `dplyr` تساعدك على إنشاء أو تعديل الأعمدة، مع الاحتفاظ بالأعمدة الموجودة.

الهيكل العام لدالة `mutate` هو:

`data %>%   mutate(new_column_name = what_it_contains)`

لنقم بتجربة `mutate` باستخدام العمود `Date` من خلال تنفيذ العمليات التالية:

1. تحويل التواريخ (التي هي حالياً من نوع نصي) إلى صيغة شهر (هذه تواريخ أمريكية، لذا الصيغة هي `MM/DD/YYYY`).

2. استخراج الشهر من التواريخ إلى عمود جديد.

في لغة R، تجعل الحزمة [lubridate](https://lubridate.tidyverse.org/) العمل مع بيانات التاريخ والوقت أسهل. لذا، دعونا نستخدم `dplyr::mutate()`، و`lubridate::mdy()`، و`lubridate::month()` لنرى كيف يمكن تحقيق الأهداف المذكورة أعلاه. يمكننا حذف عمود `Date` لأننا لن نحتاجه مرة أخرى في العمليات اللاحقة.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

واو! 🤩

الآن، دعونا ننشئ عمودًا جديدًا باسم `Price`، والذي يمثل متوسط سعر اليقطين. بعد ذلك، سنأخذ متوسط عمودي `Low Price` و`High Price` لملء العمود الجديد `Price`.


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

نعم!💪

"لكن انتظر!" ستقول بعد تصفح مجموعة البيانات بالكامل باستخدام `View(pumpkins)`, "هناك شيء غريب هنا!"🤔

إذا نظرت إلى عمود `Package`, ستجد أن القرع يُباع بتكوينات مختلفة. بعضها يُباع بمقياس `1 1/9 bushel`, وبعضها بمقياس `1/2 bushel`, وبعضها لكل قرع، وبعضها لكل رطل، وبعضها في صناديق كبيرة بأحجام مختلفة.

دعونا نتحقق من ذلك:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

رائع!👏

يبدو أن قياس وزن القرع بشكل متسق أمر صعب للغاية، لذا دعونا نقوم بتصفية القرع عن طريق اختيار فقط القرع الذي يحتوي على الكلمة *bushel* في عمود `Package` ووضع ذلك في إطار بيانات جديد `new_pumpkins`.  


#### dplyr::filter() و stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): ينشئ مجموعة فرعية من البيانات تحتوي فقط على **الصفوف** التي تلبي شروطك، وفي هذه الحالة، القرع الذي يحتوي على النص *bushel* في عمود `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): يتحقق من وجود أو غياب نمط معين في النص.

حزمة [`stringr`](https://github.com/tidyverse/stringr) توفر وظائف بسيطة لعمليات النصوص الشائعة.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

يمكنك أن ترى أننا قد حصرنا البيانات إلى حوالي 415 صفًا تحتوي على القرع بالجملة. 🤩  
<br>


#### dplyr::case_when()

**لكن انتظر! هناك شيء آخر يجب القيام به**

هل لاحظت أن كمية السلة تختلف من صف لآخر؟ تحتاج إلى توحيد التسعير بحيث تعرض السعر لكل سلة، وليس لكل 1 1/9 أو 1/2 سلة. حان الوقت لإجراء بعض العمليات الحسابية لتوحيد ذلك.

سنستخدم الدالة [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) لتعديل عمود السعر بناءً على بعض الشروط. تتيح لك `case_when` كتابة عدة عبارات `if_else()` بشكل متجه.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

الآن يمكننا تحليل التسعير لكل وحدة بناءً على قياسها بالبوشل. كل هذا البحث حول بوشل القرع يظهر مدى `أهمية` أن `تفهم طبيعة بياناتك`!

> ✅ وفقًا لموقع [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308)، يعتمد وزن البوشل على نوع المنتج، لأنه قياس للحجم. "على سبيل المثال، من المفترض أن يزن بوشل الطماطم 56 رطلاً... الأوراق والخضروات تشغل مساحة أكبر بوزن أقل، لذا فإن بوشل السبانخ يزن فقط 20 رطلاً." الأمر معقد للغاية! لن نشغل أنفسنا بتحويل البوشل إلى أرطال، وبدلاً من ذلك سنسعر بناءً على البوشل. كل هذا البحث حول بوشل القرع يظهر مدى أهمية أن تفهم طبيعة بياناتك!

> ✅ هل لاحظت أن القرع الذي يُباع بنصف بوشل مكلف جدًا؟ هل يمكنك معرفة السبب؟ تلميح: القرع الصغير أغلى بكثير من الكبير، ربما لأن هناك عددًا أكبر منه في البوشل الواحد، نظرًا للمساحة غير المستخدمة التي يشغلها قرع الفطيرة الكبير المجوف.


الآن أخيرًا، ومن باب المغامرة فقط 💁‍♀️، دعونا أيضًا ننقل عمود الشهر إلى الموضع الأول، أي "قبل" عمود "Package".

يتم استخدام `dplyr::relocate()` لتغيير مواقع الأعمدة.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

عمل رائع! 👌 لديك الآن مجموعة بيانات نظيفة ومرتبة يمكنك استخدامها لبناء نموذج الانحدار الجديد الخاص بك!  


## 4. تصور البيانات باستخدام ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>رسم توضيحي من إعداد داساني ماديبالي</figcaption>


<!--![رسم توضيحي من إعداد داساني ماديبالي](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.ar.png){width="600"}-->

هناك مقولة *حكيمة* تقول:

> "لقد قدم الرسم البياني البسيط معلومات أكثر لعقل محلل البيانات من أي أداة أخرى." --- جون توكي

جزء من دور عالم البيانات هو توضيح جودة وطبيعة البيانات التي يعمل عليها. وللقيام بذلك، يقومون غالبًا بإنشاء تصورات مثيرة للاهتمام، مثل الرسوم البيانية والمخططات التي تعرض جوانب مختلفة من البيانات. وبهذه الطريقة، يمكنهم عرض العلاقات والفجوات بصريًا التي قد يكون من الصعب اكتشافها بطرق أخرى.

يمكن أن تساعد التصورات أيضًا في تحديد تقنية التعلم الآلي الأنسب للبيانات. على سبيل المثال، إذا كان الرسم البياني النقطي يبدو وكأنه يتبع خطًا، فهذا يشير إلى أن البيانات مرشحة جيدة لتمرين الانحدار الخطي.

يوفر R العديد من الأنظمة لإنشاء الرسوم البيانية، ولكن [`ggplot2`](https://ggplot2.tidyverse.org/index.html) هو واحد من أكثرها أناقة وتنوعًا. يتيح لك `ggplot2` إنشاء الرسوم البيانية عن طريق **دمج مكونات مستقلة**.

لنبدأ بمخطط نقطي بسيط لعمودي Price وMonth.

في هذه الحالة، سنبدأ بـ [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html)، ونوفر مجموعة بيانات وتعيينًا جماليًا (باستخدام [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html))، ثم نضيف طبقات (مثل [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) لإنشاء المخططات النقطية.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

هل هذا الرسم البياني مفيد 🤷؟ هل هناك شيء فيه يثير دهشتك؟

ليس مفيدًا بشكل خاص لأنه يعرض بياناتك فقط كتناثر من النقاط في شهر معين.  
<br>


### **كيف نجعلها مفيدة؟**

لجعل الرسوم البيانية تعرض بيانات مفيدة، عادةً ما تحتاج إلى تجميع البيانات بطريقة ما. على سبيل المثال، في حالتنا، العثور على متوسط سعر القرع لكل شهر سيقدم المزيد من الأفكار حول الأنماط الأساسية في بياناتنا. هذا يقودنا إلى نظرة سريعة أخرى على **dplyr**:

#### `dplyr::group_by() %>% summarize()`

يمكن حساب التجميع المُجمّع في R بسهولة باستخدام

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` يغير وحدة التحليل من مجموعة البيانات الكاملة إلى مجموعات فردية مثل كل شهر.

-   `dplyr::summarize()` ينشئ إطار بيانات جديد يحتوي على عمود لكل متغير تجميعي وعمود لكل إحصائية تلخيصية قمت بتحديدها.

على سبيل المثال، يمكننا استخدام `dplyr::group_by() %>% summarize()` لتجميع القرع في مجموعات بناءً على أعمدة **الشهر** ثم العثور على **متوسط السعر** لكل شهر.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

موجز!✨

الميزات التصنيفية مثل الأشهر يتم تمثيلها بشكل أفضل باستخدام مخطط الأعمدة 📊. الطبقات المسؤولة عن مخططات الأعمدة هي `geom_bar()` و `geom_col()`. راجع `?geom_bar` لمعرفة المزيد.

لنقم بإنشاء واحد!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 هذه طريقة أكثر فائدة لتصور البيانات! يبدو أنها تشير إلى أن أعلى سعر للقرع يحدث في شهري سبتمبر وأكتوبر. هل يتوافق ذلك مع توقعاتك؟ لماذا أو لماذا لا؟

تهانينا على إنهاء الدرس الثاني 👏! لقد قمت بتحضير بياناتك لبناء النموذج، ثم اكتشفت المزيد من الأفكار باستخدام التصورات!



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
